Not working properly in RStudio or Visual Studio

Resort to ipython and plain text editor

This notebook is inspired by the [2nd place solution of Mechanisms of Action (MoA)<br>
Prediction](https://www.kaggle.com/c/lish-moa/discussion/202256).

This Python 3 environment comes with many helpful analytics libraries installed<br>
It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python<br>
For example, here's several helpful packages to load

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

Input data files are available in the read-only "../input/" directory<br>
For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os<br>
for dirname, _, filenames in os.walk('/kaggle/input'):<br>
    for filename in filenames:<br>
        print(os.path.join(dirname, filename))

You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" <br>
You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')<br>
test = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')

In [ ]:
dataset_train = pd.read_csv("santander/train.csv")
dataset_test = pd.read_csv("santander/test.csv")

numerical_cols = [f'cont{i}' for i in range(1, 15)]

In [ ]:
target_col = 'target'

for c in numerical_cols:<br>
    prep = StandardScaler()<br>
    train[c] = prep.fit_transform(train[[c]])<br>
    test[c] = prep.transform(test[[c]])

In [ ]:
X_train = dataset_train.drop(['ID_code', 'target'], axis=1)
y_train = dataset_train['target']
X_test = dataset_test.drop('ID_code', axis=1)

In [ ]:
X_train.head(2)
X_test.head(2)

Just for now, use X_test as validation. Sort out later

In [ ]:
X_val = X_test

cv = KFold(n_splits=5, shuffle=True, random_state=7)<br>
<br>
y_preds = []<br>
models = []<br>
oof_train = np.zeros((len(X_train),))<br>
<br>
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):<br>
    X_tr = X_train.loc[train_index, :]<br>
    X_val = X_train.loc[valid_index, :]<br>
    y_tr = y_train.loc[train_index]<br>
    y_val = y_train.loc[valid_index]

In [ ]:
model = keras.Sequential([
#   layers.BatchNormalization(),
#   layers.Dropout(0.1),
    layers.Dense(4096, activation='relu'),
    layers.Reshape((256, 16)),
#   layers.BatchNormalization(),
#   layers.Dropout(0.1),
    layers.Conv1D(filters=16, kernel_size=5, strides=1, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='linear'),
])